# Classification with Doc2Vec and SVM

It is a binary classification with two labels {'no Asshole': 0,'Asshole': 1}.

Several steps are included during the training and testing process:

1. Create feature representations for sentences

2. Use oversampling to balance two classes given features vector

3. Use balanced x_train and y_train to train SVM model

4. Test the trained model with x_test and report the evaluation results

Download the dataset

In [1]:
!pip install dvc

     |████████████████████████████████| 399 kB 5.2 MB/s 
     |████████████████████████████████| 109 kB 44.4 MB/s 
     |████████████████████████████████| 132 kB 46.1 MB/s 
     |████████████████████████████████| 296 kB 42.5 MB/s 
     |████████████████████████████████| 44 kB 2.9 MB/s 
     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
     |████████████████████████████████| 4.6 MB 17.8 MB/s 
     |████████████████████████████████| 546 kB 46.7 MB/s 
     |████████████████████████████████| 214 kB 45.7 MB/s 
     |████████████████████████████████| 180 kB 47.0 MB/s 
     |████████████████████████████████| 41 kB 37 kB/s 
     |████████████████████████████████| 1.1 MB 39.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 2.6 MB 39.5 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 222 kB 43.2 MB/s 
     |██████████████████████

In [2]:
!dvc get https://github.com/iterative/aita_dataset aita_clean.csv

Load necessary libraries

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import doc2vec
from tqdm import tqdm
from sklearn import utils
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

Data processing
* Select rows with `score > 10`, which means more than ten people make judgement for it
* Combine `title` and `body`, fill the missing cell with an empty string

In [4]:
df = pd.read_csv('aita_clean.csv')
df = df[df['score'] >= 10]
df['text'] = df['title'] + df['body'].fillna('')

In [5]:
def label_sentences(corpus, label_type):
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(str(v).split(), [label]))
    return labeled

In [6]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.is_asshole, random_state=0, 
                                                    test_size=0.3)
x_train = label_sentences(x_train, 'Train')
x_test = label_sentences(x_test, 'Test')
all_data = x_train + x_test

In [7]:
model_dbow = doc2vec.Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, 
                     min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

100%|██████████| 48853/48853 [00:00<00:00, 1709459.25it/s]


In [8]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), 
                     total_examples=len(all_data), 
                     epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 48853/48853 [00:00<00:00, 2270332.66it/s]


Create x_train and x_test vectors

In [9]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [10]:
train_vectors_dbow = get_vectors(model_dbow, len(x_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(x_test), 300, 'Test')

Use oversampling to eliminate class inbalance

In [11]:
smt = SMOTE(random_state = 42)
X_train, Y_train = smt.fit_resample(train_vectors_dbow, y_train)

In [12]:
model = SVC(C=10, gamma='auto', kernel='rbf')
model.fit(X_train, Y_train)

SVC(C=10, gamma='auto')

Generate classification report

In [13]:
y_pred = model.predict(test_vectors_dbow)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.7199099344978166
              precision    recall  f1-score   support

           0       0.73      0.96      0.83     10623
           1       0.45      0.08      0.14      4033

    accuracy                           0.72     14656
   macro avg       0.59      0.52      0.49     14656
weighted avg       0.66      0.72      0.64     14656

